In [22]:
import pandas as pd
import pandas_ta as ta
import glob
from datetime import datetime, time

headers = ['Date','Time','Open','High','Low','Close','Volume']
path = "C:\\Users\\desma\\Downloads\\jupyter work\\DATA\\"
#set up so i could run multiple datasets if wanted
filenames = glob.glob(path + "TSLA5m.txt")
for file in filenames:
    print("\nReading file = ", file)
    df = pd.read_csv(file, header = None)
    df.columns=headers
dfLen = len(df)
dfLen


Reading file =  C:\Users\desma\Downloads\jupyter work\DATA\TSLA5m.txt


49843

In [23]:
#have to make levels and signal for each day. no overnights
from datetime import datetime, time
import numpy as np


openStr = '9:30'
closeStr = '16:00'
mktOpen = datetime.strptime(openStr, '%H:%M')
mktClose = datetime.strptime(closeStr, '%H:%M')
high, low, mOpen, mClose = None, None, None, None
signal = [0]*dfLen



def preLvls(close, open):  #row values, indexes
    low = np.inf
    high = -np.inf
    out = [None, None]
    for x in range(close+2, open):  #same strat as before for finding extrema, just shorter interval
        currentH = df.High[x-1]
        currentL = df.Low[x-1]
        if df.High[x-2] < currentH > df.High[x]:
            high = max(high, currentH)
            out[1] = high
        elif df.Low[x-2] > currentL < df.Low[x]:
            low = min(low, currentL)
            out[0] = low
    
        
    return out


for row in range(dfLen):
    now = df.Time[row]
    currTime = datetime.strptime(now, '%H:%M')
    if currTime == mktClose: #reset prev day values
        mClose = row
        high, low = None, None
    elif currTime == mktOpen:
        mOpen = row
        if mClose and mOpen: #saftey precaution to make sure I have both values 
            lvls = preLvls(mClose, mOpen)  #now have current idx bounds to find extrema of AH/Pre levels
            low, high = lvls[0], lvls[1]
            continue  #go to next iteration, since currently 930 need another row to start comp B/S
            
    if high and low: #mkt open, have ran thru full AH/PM rows and have extrema (if exist)
        if df.Close[row] > high and df.Close[row-1] < high:  
            signal[row] = 1
        elif df.Close[row] < low and df.Close[row-1] > low:
            signal[row] = 2
    
df['Signal'] = signal

In [24]:
df['Signal'].value_counts()

0    49092
1      385
2      366
Name: Signal, dtype: int64

In [25]:
def SIGNAL():
    return df.Signal

In [30]:
from backtesting import Strategy, Backtest
import numpy as np

class myStrat(Strategy):
    def init(self):
        super().init()
        self.signal1=self.I(SIGNAL)     
        
    def next(self):
        super().next()
        trailSL = self.data.Close[-1]*.025
        for trade in self.trades:
            if trade.is_long:
                trade.sl = max(trade.sl or -np.inf, self.data.Close[-1] - trailSL)
            else:
                trade.sl = min(trade.sl or np.inf, self.data.Close[-1] + trailSL)
        
        if self.signal1==1:# and len(self.trades)==0: #Bullish Signal
            self.buy()#, tp=tp1)
            
        elif self.signal1==2:# and len(self.trades)==0: #Bearish Signal
            self.sell()#, tp=tp2) 
            
bt = Backtest(df, myStrat, cash=10_000, margin = 1/1, commission=.000)
stat = bt.run()
stat
#bt.plot()

C:\Users\desma\AppData\Local\Temp/ipykernel_11940/646858174.py:24: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df, myStrat, cash=10_000, margin = 1/1, commission=.000)


Start                                     0.0
End                                   49842.0
Duration                              49842.0
Exposure Time [%]                   61.587384
Equity Final [$]                  9707.535507
Equity Peak [$]                  12229.938128
Return [%]                          -2.924645
Buy & Hold Return [%]               10.401779
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -32.182928
Avg. Drawdown [%]                   -2.905227
Max. Drawdown Duration                19124.0
Avg. Drawdown Duration                 1239.5
# Trades                                266.0
Win Rate [%]                        36.466165
Best Trade [%]                      12.793709
Worst Trade [%]                     -5.533663
Avg. Trade [%]                    